In [2]:
from bs4 import BeautifulSoup
import urllib
import re
#general format is https://www.kickstarter.com/projects/creatorname/campaignname/description
r = urllib.urlopen('https://www.kickstarter.com/projects/802159142/nuimo-seamless-smart-home-interface?ref=discover_potd')
soup = BeautifulSoup(r)


In [3]:
title= soup.find_all("meta", property="og:title")[0].get("content")
title

'Nuimo: Seamless Smart Home Interface'

In [3]:

blurb=soup.find_all(property=re.compile("og:description"))[0].get("content")
blurb

'ENVELOP is a 3D sound platform that is creating a space in San Francisco for live electronic music, DJ sets and educational workshops.'

In [4]:
creator_url=soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
creator_url

'https://www.kickstarter.com/profile/envelop'

In [50]:

a=soup.find_all("div", class_="num h1 bold")[0]
no_backers=a.contents[0].string
print no_backers

285


In [107]:
#reg = re.compile("\d.*",flags=re.U)
#goal=reg.findall(ist(soup.find_all("span", class_="money usd no-code")[2])[0].string)
goal= soup.find_all("div", class_="num h1 bold nowrap")[0].get("data-goal")
goal

'27333.0'

In [112]:
#no_dollars_raised=list(soup("div", class_="NS_projects__stats")[0])[3].string
#no_dollars_raised
no_dollars_raised= soup.find_all("div", class_="num h1 bold nowrap")[0].get("data-pledged")
no_dollars_raised

'29303.0'

In [161]:
#length of campaign
x=[item.get("data-duration")  for item in soup.find_all("span") ]
campaign_duration=[item for item in x if item is not None][0]
y=[item.get("data-hours-remaining")  for item in soup.find_all("span") ]
campaign_hours_remaining=[item for item in y if item is not None][0]
print campaign_duration
print campaign_hours_remaining

33.0
381.7912868479888


In [52]:
def something(b):
    try:
        a = 10 / b
    except:
        a = None
    return(a)

something(0)

In [347]:

b=list(set(list(soup.find_all(class_="NS_backer_rewards__reward p2"))))


#item("span", class_="limited-number")[0].get_text()
#[item("span", class_="limited-number")[0].get_text() for item in b]
limited = []
for item in b: 
    try:
        limited.append(item("span", class_="limited-number")[0].get_text())
    except:
        limited.append(None)
        
limited[0]

u'Limited (67 left of 75)'

In [348]:
#Rewards
backer_list = [backers("span", class_="num-backers mr1")[0].get_text().replace("\n","") for backers in b]
reward_item_list = list([item("div",class_="desc h5 mb2 break-word")[0].get_text().replace("\n","") for item in b])
reg = re.compile("\d+",flags=re.U)
reward_value_list=[reg.findall(item("h5",class_="mb1")[0].span.get_text().replace(",","")) for item in b]

print backer_list[0]
print reward_value_list[0]
print reward_item_list[0]

8 backers
[u'25']
Single-ticket voucher to and ENVELOP event1 ticket promo code voucher valid for ENVELOP-produced events and performances. This voucher can be used in lieu of payment for a single event ticket. Not valid for special events or 3rd party productions. (Our events will be awesome.)


In [345]:
reward_backer_tup = (zip(reward_value_list, backer_list,reward_item_list, limited))
reward_backer_tup[0]

([u'25'],
 u'8 backers',
 u'Single-ticket voucher to and ENVELOP event1 ticket promo code voucher valid for ENVELOP-produced events and performances. This voucher can be used in lieu of payment for a single event ticket. Not valid for special events or 3rd party productions. (Our events will be awesome.)',
 u'Limited (67 left of 75)')

In [5]:
full_description=soup("div", class_="full-description js-full-description responsive-media formatted-lists")[0].find_all("p")
full_description

[<p>Nuimo is an intuitive and natural way to interact with your connected devices. Nuimo works with any bluetooth device or application including Sonos and Philips Hue. </p>,
 <p>Nuimo is a freely programmable controller and wireless so you can take it anywhere. Using the nuimo you can control all of your devices through our simple and seamless physical controller.</p>,
 <p>Unlike the touch screen, nuimo has a number of touch based inputs that feel familiar and suit your needs. It incorporates capacitive touch, gesture recognition and a 360 degree analog ring that gives you precise control over everything from the volume of your music to switching off your lights. These inputs are easily mappable to the devices and applications you care about most. </p>,
 <p> The LED matrix on the face of the device lets you change apps quickly or event display a timer app in the kitchen. </p>,
 <p>We have built 30+ integrations for nuimo and with our community of developers, that number is growing rap

In [350]:
c=list(soup("div", class_="mb6 mb2"))
r=[item("p") for item in c][0]

#Grabbing the risk text section
risk=[para.get_text() for para in r]
risk

[u"The only main risk is that our launch date might be delayed because we are subject to The Midway's construction schedule (The Midway is the larger complex that the ENVELOP space is a part of). Currently our room is being built, see photo above in the budget section. If there are any construction delays with the room we are installing the system in, our launch date will be delayed as well.",
 u'Another challenge is making sure people know that ENVELOP is more than just a venue in San Francisco. It\u2019s a platform for 3D immersive sound and anyone in the world can benefit from this technology.']

In [63]:
import urllib2
import json

In [65]:
def get_one_project_det(pid, url, currency):
    r = urllib.urlopen(url)
    soup = BeautifulSoup(r)
    
    #the reward values and rewards
    b=list(soup.find_all(class_="NS_projects__rewards_list js-project-rewards"))
    reward_value_list = [reward.string.split()[0] for reward in b[0]("h5",class_="mb1")]
    backer_list = [backers.string.split()[0]+' backers' for backers in b[0]("span", class_="num-backers mr1")]
    reward_item_list = [item("p")[0].get_text() for item in b[0]("div",class_="desc h5 mb2 break-word")]
    
    #removing currency sign from reward value
    reg = re.compile("\d.*",flags=re.U)
    reward_value_list = [reg.findall(reward.string.split()[0])[0] for reward in b[0]("h5",class_="mb1")]

    
    
    #risk information
    c=list(soup("div", class_="mb6 mb2"))
    risk=[item("p") for item in c][0]
    
    
    project_det = dict()
    project_det["title"] = soup.find_all("meta", property="og:title")[0].get("content")
    #project_det["blurb1"] = soup.find_all(property=re.compile("og:description"))[0].get("content") 
    project_det["creator_url"] = soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
    project_det["no_backers"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[1].string
    project_det["goal"] = reg.findall(list(soup.find_all("span", class_="money " +currency.lower()+ " no-code")[2])[0].string)
    project_det["no_dollars_raised"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[3].string
    project_det["reward_backer_tup"] = (zip(reward_value_list, backer_list,reward_item_list))
    project_det["risk"] = [para.get_text() for para in risk]
    project_det["id"] = pid
    project_det["url"] = url
    project_det["description"]=soup("div", class_="full-description js-full-description responsive-media formatted-lists")[0].find_all("p")
    x=[item.get("data-duration")  for item in soup.find_all("span") ]
    project_det["campaign_duration"]=[item for item in x if item is not None][0]
    y=[item.get("data-hours-remaining")  for item in soup.find_all("span") ]
    project_det["campaign_hours_remaining"]=[item for item in y if item is not None][0]
   
    return(project_det)

In [66]:
#open url information
fd = open('./data/project_successful_meta.json', 'r')
text = fd.read()
fd.close()

pmeta = json.loads(text)

In [68]:
project_live_det = [get_one_project_det(project_header["id"], project_header["url"],project_header["currency"]) for project_header in pmeta[0:500]]

In [ ]:
from bs4 import BeautifulSoup
import urllib
import re

import urllib2
import json

#Save data to file for further use
project_meta_json = json.dumps(obj = project_succ_det, indent = 4)
fd = open("./data/project_live_detail"++".json", "w")
fd.write(project_meta_json)
fd.close()

In [60]:
reward_item_list[1]

u'EARLY BIRD: Receive one Pantelligent pan at a special discount. Thank you for being among the first to support our vision!\n(Please add $30 for international shipping)'